In [1]:
import pandas as pd
import os
import glob
import concurrent.futures
import time

directory = 'C:\\Users\\Trieu Pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
directory2 = 'C:\\Users\\trieu.pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
cate_dir = 'C:\\Users\\Trieu Pham\\Desktop\\DataCate.xlsx'
cate_dir2 = 'C:\\Users\\trieu.pham\\Desktop\\DataCate.xlsx'

# list of Excel files to read data from
if os.path.exists(os.path.join(directory, '230101 Payment Summary - 2023-01-03T093340.600.xlsx')):
    excel_files = glob.glob(os.path.join(directory, '*.xlsx'))
else:
    excel_files = glob.glob(os.path.join(directory2, '*.xlsx'))
try:
    # use cols 0, Barcode; 2, Category Name
    # df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,2])
    # use cols 0, Barcode; 3, SubCategory Name
    df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,3])
    pass
except FileNotFoundError:
    # df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,2])
    # use cols 0, Barcode; 3, SubCategory Name
    df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,3])
    pass
# create an empty list to store the dataframes
# dfs = [] sho

outputfile =  'dataT!.csv'
# def excel2csv
def excel2csv(inputfile, outputfile):
    
    # Check if exist file_path
    if not os.path.exists(outputfile):
        with open(outputfile, 'w') as f:
            pass

    # Read the first line of the file
    with open(outputfile, 'r') as f:
        first_line = f.readline().strip()
    # Check if the first line contains the expected column names
    has_headers = 'InvoiceID' in first_line
    df = pd.read_excel(outputfile, sheet_name='Sheet2', skiprows=2, usecols=[4,7,8,14])
    # df2 Category
    
    # merge with Category, on "Barcode"
    df = df.merge(df2, on='Barcode', how='left')
    
    # Check if the first line contains the expected column names
    df.to_csv(outputfile, mode='a', header= not has_headers, index=False)


# loop through the list of Excel files
with concurrent.futures.ThreadPoolExecutor() as executor:
    [executor.submit(excel2csv, file, outputfile) for file in excel_files]
    # for file in excel_files:
    #     executor.submit(excel2csv,file)

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Sample data
df1 = pd.read_csv('data.csv')


# Get data from Invoice ID with Barcode
df1 = df1[['InvoiceID','SubCategory Name']]

df1 = df1[df1.groupby('InvoiceID')['SubCategory Name'].transform('size') > 1]
dataset_str = df1
dataset_str = df1.groupby('InvoiceID').agg({
    'SubCategory Name': lambda x: ','.join(str(i) for i in x)
}).reset_index()

dataset_str

C:\Users\trieu.pham\AppData\Local\Temp\ipykernel_8608\1094835795.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('data.csv')


,InvoiceID,SubCategory Name
0,InvoiceID,"SubCategory Name,SubCategory Name,SubCategory ..."
1,VN0001010101230003,"Other Ice Cream,Potato Snacks,Stick,UHT Milk,Mask"
2,VN0001010101230004,"Distilled Spirits,Cup Of Ice,Add-on item Sub,D..."
3,VN0001010101230005,"Sports Drinks Sub,Sports Drinks Sub"
4,VN0001010101230006,"Sports Drinks Sub,Others Drinks,Mineral water,..."
...,...,...
3682574,VN9996023101230486,"Sports Drinks Sub,Sports Drinks Sub,Energy Dri..."
3682575,VN9996023101230488,"Gimbap,Cup Of Ice,Energy Drinks Sub"
3682576,VN9996023101230489,"Potato Snacks,Cup Of Ice,Distilled Spirits"
3682577,VN9996023101230492,"Domestic Cigarettes,Cup Of Ice"


In [5]:
# Transform the data into a list of lists (transactions) with TransactionEncoder
# create a TransactionEncoder object
data = dataset_str[['SubCategory Name']]

data = data.values.tolist()

data = [x[0].split(',') for x in data]

te = TransactionEncoder()

# transform the data into a list of lists
transactions = te.fit(data).transform(data)
transactions = pd.DataFrame(transactions, columns=te.columns_)

transactions

,Add-on item Sub,Alternative Snacks,Babycare sub,Batteries Sub,Beauty Hair & Nail,Bento,Bingsu,Body Care,Can food,Cereal,...,Toys Candy,UHT Milk,Vegetables Sub,Vegetarian food,WIC/RIC Juice,Wafflers,Water/Beverage Enhancers,Wearables/Apparel/Umbrella,Western Coffee,nan
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682574,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682575,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682576,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682577,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
# Calculate frequent itemsets using Apriori algorithm

frequent_itemsets = apriori(transactions, min_support=0.005, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.007401,(Add-on item Sub)
1,0.019209,(Alternative Snacks)
2,0.036771,(Bento)
3,0.014042,(Chilled Ready to eat Sub)
4,0.032282,(Chocolate Bar)
...,...,...
140,0.005366,"(Sugar CSD, Salty Snacks)"
141,0.011166,"(Sugar Tea, Salty Snacks)"
142,0.005125,"(Sugar Tea, Sandwich)"
143,0.009103,"(Sugar CSD, Sugar Tea)"


In [12]:
# Generate association rules from the frequent itemsets

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bento),(Onigiri),0.036771,0.103501,0.006146,0.167143,1.614891,0.002340,1.076414
1,(Onigiri),(Bento),0.103501,0.036771,0.006146,0.059381,1.614891,0.002340,1.024037
2,(Sugar Tea),(Bento),0.156438,0.036771,0.006166,0.039415,1.071925,0.000414,1.002753
3,(Bento),(Sugar Tea),0.036771,0.156438,0.006166,0.167689,1.071925,0.000414,1.013519
4,(Cup Of Ice),(Energy Drinks Sub),0.083932,0.080919,0.021213,0.252744,3.123441,0.014422,1.229942
...,...,...,...,...,...,...,...,...,...
111,(Sandwich),(Sugar Tea),0.039010,0.156438,0.005125,0.131382,0.839839,-0.000977,0.971155
112,(Sugar CSD),(Sugar Tea),0.072158,0.156438,0.009103,0.126148,0.806377,-0.002186,0.965338
113,(Sugar Tea),(Sugar CSD),0.156438,0.072158,0.009103,0.058187,0.806377,-0.002186,0.985165
114,(Sugar Tea),(WIC/RIC Juice),0.156438,0.037814,0.009855,0.062998,1.666020,0.003940,1.026878


In [3]:
# Display the association rules with support, lift, and confidence
import time
print(rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']])
rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']].to_csv('result'+ time.localtime() + '.csv', index=False)

NameError: name 'rules' is not defined